In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

createDummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))



In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_2544/3753290993.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


In [7]:
# prover calculates settings, send to verifier
# In non-linearity, scale greatly affects lookup size
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        # self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


prover_gen_settings(data_path,['col_name'], sel_data_path, prover_model,prover_model_path, [8], "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[0,593654],"logrows":20,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":905,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,16],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[{"Recip":{"scale":256.0}},{"Ln":{"scale":256.0}},{"Exp":{"scale":65536.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_2544/1886633819.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
# We can see here that it took huge resource, yet still inaccurate
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 219.55537796020508 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 92.68629455566406
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9340507686675262708, 12576478094795452846, 16595392862476743742, 1007129139500821923]]], 'proof': '22df37dac0ac9a5c8f99b34e177d0f83bb847512c7948f08bb6838f1d3d2813823f76704dfe20fd5b52753439a3f031e5f85df8e1eacf08f4bfdefbe01a7fa3b2a0303d3ccf17a6499fa4008ab8749776bfbc1e5d60bad5f67980e80b5690a9e0387e223ebea43741d97104c12f821daa0b63c1c7e4f783117bc8d16ac7579eb08cdbbc4a24a325c90359792c8a6b2cdc24e972c204e0aa1aac14210a386649d0bb19842e11022a538be33391b0b16907190850fe7d7ee4aabeaa0a8899fb39f1113171d155bd30679963c6fdcc2fda33e346c59027538ddb05ee49d93af544a25b98cac646d67bfd83d357f15c6934fc08dc49952e00365b40cff83c991fac52c0905b243126ada9653b49bc01c0aa9b1846b7038af7415d77c8796181ada6102823f7e380b84b3379337e23cec47a3fbe33b9db9b373791ca0822c9e5fa40e1218495cfc6f04b4650cc4f7e1e631b9f090001efcd3409b0cc3c7130b8199

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9340507686675262708, 12576478094795452846, 16595392862476743742, 1007129139500821923]]]
proof boolean:  1.0
proof result 1 : 92.68629455566406
verified
